In [1]:
import os
import logging
import json

from dotenv import load_dotenv
from datetime import datetime

from df_processing import user_df, org_group_description, user_report_df, report_df_processing, datatype_conversion

from core.jinja_templating import SQLTemplateLoader
from core.excel_utils import cool_write_function, empty_directory
from core.sftp_management import SFTPManager

In [2]:
# loading .env file 
load_dotenv()
logger = logging.getLogger(__name__)

In [3]:
## adjust this to your user path
user_path = "/Users/blayton"

with open(f"{user_path}/Documents/solution_engineering/cc_reports/cc_intake_reporting/config.json", 'r') as file:
    data = json.load(file)  

In [4]:
organization_id = data["organization_id"]
customer_abbreviation = data["customer_abbreviation"]
output_directory_base=data["output_directory"]
sql_template_path_base=data["template_directory"]

output_directory=f"{user_path}/{output_directory_base}"
sql_template_path=f"{user_path}/{sql_template_path_base}"

In [5]:
## rendering sql templates using jinja
if __name__ == "__main__":
    sql_loader = SQLTemplateLoader(sql_template_path)

    rendered = sql_loader.render_all_templates(**data)


Loaded 8 SQL templates


In [6]:
def report_generation(**kwargs):
    start_time = datetime.now()

    users_df = user_df(kwargs['users_report'])
    users_for_reports_df = users_df.copy()
    del kwargs['users_report']

    #clears the existing output directory
    empty_directory(output_directory)

    for key, query in kwargs.items():
        if key == 'org_group_description':  # Check if the current path is the org_group_path
            org_df = org_group_description(query)
            key = 'users_report'
            df = user_report_df(org_df, users_for_reports_df)

        elif key == 'document_inventory_by_contract':
            df = report_df_processing(users_df, query)
        
        elif key == 'date_mistmatch':
            df = report_df_processing(users_df, query)

        else:  # For all other keys
            df = report_df_processing(users_df, query)

        df = datatype_conversion(df)

        # Write the resulting dataframe to a workbook
        cool_write_function(customer_abbreviation, output_directory, key, df)
        logger.info(f"{key} component processed.")

    end_time = datetime.now()
    execution_time = end_time - start_time

    logger.info(f"Reports generated: {execution_time} seconds")

In [7]:
def sftp():
    sftp_creds = data['sftp_creds']
    sftp_host = sftp_creds.get('sftp_host')
    sftp_port = sftp_creds.get('sftp_port')
    remote_output = sftp_creds.get('remote_output')
    parent_directory = sftp_creds.get('parent_directory')
    local_path_base = sftp_creds.get('local_path')
    local_path = f"{user_path}/{local_path_base}"

    all_file_paths = []

    # prepping output path
    current_date = datetime.now().strftime('%Y-%m-%d')
    remote_path = os.path.join(remote_output, parent_directory, current_date)
        
    logging.info(f"Traversing directory: {local_path}")
    
    for root, dirs, files in os.walk(local_path):
        for file in files:
            # Create the full path by joining the root and filename
            full_path = os.path.join(root, file)
            
            all_file_paths.append(full_path)

    sftp_username = os.getenv('CSH_SFTP_USERNAME') 
    sftp_password = os.getenv('CSH_SFTP_PASSWORD') 

    sftp_manager = SFTPManager()
    sftp_manager.connect(hostname=sftp_host, port=sftp_port, username=sftp_username, password=sftp_password)

    # Building file path, parent_directory is specific to the type of report based on config.json
    sftp_manager.mkdir_p(remote_path)
            
    for local_file in all_file_paths:
        filename = os.path.basename(local_file)
        file_remote_path = os.path.join(remote_path, filename)

        sftp_manager.upload_file(local_file, file_remote_path)
    
    sftp_manager.disconnect()

In [8]:
report_generation(**rendered)

INFO:core.query_utils.query_utils:Running query for DataFrame: SELECT au.id,
    au.first_name,
    au.last_name,
    au.email,
    ou.department,
    STRING_AGG(CONCAT(org.role, ' - ', op.name), ', ') AS cc_role_permissions,
    ou.role AS user_title,
    oog.name AS group_name,
    au.last_login,
    oogou.organization_group_id
FROM auth_user au
    INNER JOIN tq_organizations_organizationuser ou ON au.id = ou.user_id
    LEFT JOIN tq_organizations_organizationuser_groups ooug ON ooug.organizationuser_id = ou.id
    LEFT JOIN tq_organizations_rolegroup org ON org.group_ptr_id = ooug.group_id
    LEFT JOIN tq_organizations_product op ON op.id = org.product_id
    LEFT JOIN tq_organizations_organizationgrouporganizationuser oogou ON oogou.organization_user_id = ou.id
    LEFT JOIN tq_organizations_organizationgroup oog ON oog.organization_id = ou.organization_id
    AND oog.id = oogou.organization_group_id
WHERE (ou.organization_id = '28'
    AND au.is_staff != 'TRUE'
    AND ou.delete

     contract_external_id  contract_id state  \
0            65891.121910        25724    AR   
1            65891.122708        26052    AR   
2            65891.124474        25949    AR   
3            65891.148518        25950    AR   
4            65891.148886        26062    AR   
...                   ...          ...   ...   
4092                 None        58513    CA   
4093                 None        58515    CA   
4094                 None        58517    CA   
4095                 None        58518    CA   
4096                 None        58519    CA   

                                          contract_name  \
0      Amerigroup_AR_Arkansas Health Network_11/17/2017   
1      QualChoice_AR_Arkansas Health Network_01/01/2017   
2            BCBS_AR_Arkansas Health Network_06/01/2018   
3            BCBS_AR_Arkansas Health Network_01/01/2024   
4     The Stellar Health Group, Inc_AR_Arkansas Heal...   
...                                                 ...   
4092  Blue

INFO:__main__:contract_inventory component processed.


File saved to /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_CONTRACT_INVENTORY_20250606.xlsx


INFO:core.query_utils.query_utils:Running query for DataFrame: SELECT organization_group_id,
    STRING_AGG(permissions, ', ') AS group_permission_details
FROM (
        SELECT organization_group_id,
            CASE
                WHEN all_payers_access THEN 'All Payer Access'
                ELSE NULL
            END AS permissions
        FROM contracting_platform_clearcontractsscope
        UNION ALL
        SELECT organization_group_id,
            CASE
                WHEN all_providers_access THEN 'All Providers Access'
                ELSE NULL
            END AS permissions
        FROM contracting_platform_clearcontractsscope
        UNION ALL
        SELECT organization_group_id,
            CASE
                WHEN all_provider_entities_access THEN 'All Provider Entities Access'
                ELSE NULL
            END AS permissions
        FROM contracting_platform_clearcontractsscope
        UNION ALL
        SELECT organization_group_id,
            CASE
            

...

                                                  name
0                       National - CommonSpirit Health
1    CA - North State Clinics - Partnership HP of C...
2                 CA - Dignity Health Connected Living
3                WA - Virginia Mason Franciscan Health
4                    CA - Yolo Adult Day Health Center
..                                                 ...
224                             WA - St. Anne Hospital
225                    WA - St. Michael Medical Center
226                 WA - Virginia Mason Medical Center
227                              WI - Franciscan Villa
228                 CA - Citrus Heights Surgery Center

[229 rows x 1 columns]
File saved to /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_STAKEHOLDERS_PROVIDER_ENTITY_20250606.xlsx


INFO:core.query_utils.query_utils:Running query for DataFrame: --CTE to pull in related contracts
with relatedcontract as 
		(
		select conproj.originating_organization_id as orgid
			, lna.legacycontractproject_ptr_id as from_contract_id
			, conproj."name" as from_contract_name
			, lna_rc.to_legacynetworkagreement_id as to_contract_id
			, conproj2."name" as to_contract_name
		from contracting_platform_legacynetworkagreement lna 
		inner join contracting_platform_legacynetworkagreement_related_contracts lna_rc  
			on lna.legacycontractproject_ptr_id = lna_rc.from_legacynetworkagreement_id 
		inner join contracting_platform_legacynetworkagreement lna_rc2
			on lna_rc.to_legacynetworkagreement_id = lna_rc2.legacycontractproject_ptr_id 
		inner join contracting_platform_contractproject conproj 
			on conproj.id = lna.legacycontractproject_ptr_id 
		inner join contracting_platform_contractproject conproj2
			on conproj2.id = lna_rc2.legacycontractproject_ptr_id 
		)
		
--master invento

      contract_external_id  contract_id state  \
0             65891.121910        25724    AR   
1             65891.122708        26052    AR   
2             65891.124474        25949    AR   
3             65891.148518        25950    AR   
4             65891.148886        26062    AR   
...                    ...          ...   ...   
10305                 None        58513    CA   
10306                 None        58515    CA   
10307                 None        58517    CA   
10308                 None        58518    CA   
10309                 None        58519    CA   

                                           contract_name  \
0       Amerigroup_AR_Arkansas Health Network_11/17/2017   
1       QualChoice_AR_Arkansas Health Network_01/01/2017   
2             BCBS_AR_Arkansas Health Network_06/01/2018   
3             BCBS_AR_Arkansas Health Network_01/01/2024   
4      The Stellar Health Group, Inc_AR_Arkansas Heal...   
...                                                

INFO:__main__:contract_inventory_by_provider component processed.
INFO:__main__:Reports generated: 0:02:15.528566 seconds


File saved to /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_CONTRACT_INVENTORY_BY_PROVIDER_20250606.xlsx


In [9]:
sftp()

INFO:root:Traversing directory: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output
INFO:paramiko.transport:Connected (version 2.0, client AWS_SFTP_1.2)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [10]:
empty_directory(output_directory)

Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_CONTRACT_INVENTORY_BY_PROVIDER_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_STAKEHOLDERS_PROVIDERS_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_DATE_MISMATCH_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_CONTRACT_INVENTORY_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_STAKEHOLDERS_PROVIDER_ENTITY_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_DOCUMENT_INVENTORY_BY_CONTRACT_20250606.xlsx
Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_intake_reporting/output/CC_CSH_USERS_REPORT_20250606.xlsx
